In [3]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
#import qgrid

In [4]:
tree

NameError: name 'tree' is not defined

In [5]:
wd = 'C://Users//17800//Desktop//python quant//Notes//pydata-book//examples'

## GroupBy Mechanics

In [6]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1':np.random.randn(5),
                   'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.054101,1.952028
1,a,two,-1.034782,-1.034096
2,b,one,0.597942,0.505307
3,b,two,-0.423225,-0.159151
4,a,one,-0.841245,-1.189383


In [7]:
123

123

In [8]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [9]:
grouped.mean()

key1
a   -0.643376
b    0.087358
Name: data1, dtype: float64

In [10]:
df['data1'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     one    -0.447673
      two    -1.034782
b     one     0.597942
      two    -0.423225
Name: data1, dtype: float64

In [11]:
df['data1'].groupby([df['key1'],df['key2']]).mean().unstack()

key2,one,two
key1,,
a,-0.447673,-1.034782
b,0.597942,-0.423225


In [12]:
#事实上，groupby的keys不必在df里，只需要保持长度相同即可
states = pd.Series(['Ohio','California','California','Ohio','Ohio'])
years = pd.Series([2005,2005,2006,2005,2006])

In [13]:
df['data1'].groupby([states,years]).mean()

California  2005   -1.034782
            2006    0.597942
Ohio        2005   -0.238663
            2006   -0.841245
Name: data1, dtype: float64

In [14]:
#grouby操作对于一个DataFrame也是成立的
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.447673  0.381322
     two  -1.034782 -1.034096
b    one   0.597942  0.505307
     two  -0.423225 -0.159151

### Iterating Over Groups

In [15]:
for name,group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.054101  1.952028
1    a  two -1.034782 -1.034096
4    a  one -0.841245 -1.189383
b
  key1 key2     data1     data2
2    b  one  0.597942  0.505307
3    b  two -0.423225 -0.159151


In [16]:
for (k1,k2), group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.054101  1.952028
4    a  one -0.841245 -1.189383
('a', 'two')
  key1 key2     data1     data2
1    a  two -1.034782 -1.034096
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.597942  0.505307
('b', 'two')
  key1 key2     data1     data2
3    b  two -0.423225 -0.159151


In [17]:
dict(list(df.groupby('key1'))) #存为一个字典

{'a':   key1 key2     data1     data2
 0    a  one -0.054101  1.952028
 1    a  two -1.034782 -1.034096
 4    a  one -0.841245 -1.189383,
 'b':   key1 key2     data1     data2
 2    b  one  0.597942  0.505307
 3    b  two -0.423225 -0.159151}

In [18]:
dict(list(df.groupby(['key1','key2'])))

{('a',
  'one'):   key1 key2     data1     data2
 0    a  one -0.054101  1.952028
 4    a  one -0.841245 -1.189383,
 ('a',
  'two'):   key1 key2     data1     data2
 1    a  two -1.034782 -1.034096,
 ('b',
  'one'):   key1 key2     data1     data2
 2    b  one  0.597942  0.505307,
 ('b',
  'two'):   key1 key2     data1     data2
 3    b  two -0.423225 -0.159151}

In [19]:
dict(list(df.groupby(['key1','key2'])))[('a','one')] #字典中的元素为DataFrame

,key1,key2,data1,data2
0,a,one,-0.054101,1.952028
4,a,one,-0.841245,-1.189383


### Selecting a Column or Subset of Columns

In [20]:
df.groupby('key1')['data1'].mean()

key1
a   -0.643376
b    0.087358
Name: data1, dtype: float64

### Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Let's consider another example DataFrame.

In [21]:
people = pd.DataFrame(np.random.randn(5,5),
                      columns=['a','b','c','d','e'],
                      index=['Joe','Steve','Wes','Jim','Travis'])
people.iloc[2,[1,2]] = np.nan
people

,a,b,c,d,e
Joe,0.008401,1.330627,-0.327951,0.857173,-0.931826
Steve,0.241014,-0.449027,-2.201659,1.136633,-0.093613
Wes,-2.103737,NaN,NaN,1.622033,-0.004108
Jim,2.253610,0.537715,-0.665453,0.847378,1.348488
Travis,0.412551,1.090390,1.489853,-1.517511,0.612355


In [22]:
mapping = {'a': 'red',
           'b': 'red',
           'c': 'blue',
           'd': 'blue',
           'e': 'red',
           'f': 'orange'}

In [23]:
people.groupby(mapping,axis=1).sum()

,blue,red
Joe,0.529222,0.407202
Steve,-1.065025,-0.301626
Wes,1.622033,-2.107845
Jim,0.181925,4.139814
Travis,-0.027658,2.115296


### Grouping with Functions

1.Using Python functions is a more generic way of defining a group mapping compared with a dict or Series.   
  
2.Any function passed as a group key will be called once per index vlaue per index value, with the return values being  used as the groups names.


In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,0.158274,1.868343,-0.993403,3.326583,0.412554
5,0.241014,-0.449027,-2.201659,1.136633,-0.093613
6,0.412551,1.090390,1.489853,-1.517511,0.612355


In [25]:
key_list = ['one','two','one','two','one']
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one -2.103737  1.330627 -0.327951  0.857173 -0.931826
  two  2.253610  0.537715 -0.665453  0.847378  1.348488
5 two  0.241014 -0.449027 -2.201659  1.136633 -0.093613
6 one  0.412551  1.090390  1.489853 -1.517511  0.612355

### Grouping by Index Levels

In [26]:
hier_df = pd.DataFrame(np.random.randn(4,5),
                        # defalut: range(4)
                        columns=[['US','US','US','JP','JP'],
                                 [1,3,5,1,3]])
hier_df.columns.names = ['cty','tenor']
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.563015 -0.353115  1.537168 -0.703040  1.162856
1      0.372622 -0.169832 -0.453216  1.909391 -1.227577
2     -1.144871 -0.089351 -0.663161  1.350338  1.297297
3      1.569475  0.614194  0.939797 -1.540215 -0.052978

In [27]:
hier_df.groupby(level='cty',axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

### Multiple Function Application

In [28]:
df

,key1,key2,data1,data2
0,a,one,-0.054101,1.952028
1,a,two,-1.034782,-1.034096
2,b,one,0.597942,0.505307
3,b,two,-0.423225,-0.159151
4,a,one,-0.841245,-1.189383


In [29]:
df.groupby('key1').quantile(0.9)

C:\Users\17800\AppData\Local\Temp/ipykernel_4664/194176330.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.quantile is deprecated. In a future version, a TypeError will be raised. Before calling .quantile, select only columns which should be valid for the function.
  df.groupby('key1').quantile(0.9)


,data1,data2
key1,,
a,-0.211530,1.354803
b,0.495825,0.438861


In [30]:
df.groupby(['key1','key2']).aggregate(lambda x:max(x)-min(x))

data1     data2
key1 key2                    
a    one   0.787144  3.141412
     two   0.000000  0.000000
b    one   0.000000  0.000000
     two   0.000000  0.000000

In [31]:
df.groupby(['key1','key2']).describe()

data1                                                              \
          count      mean       std       min       25%       50%       75%   
key1 key2                                                                     
a    one    2.0 -0.447673  0.556595 -0.841245 -0.644459 -0.447673 -0.250887   
     two    1.0 -1.034782       NaN -1.034782 -1.034782 -1.034782 -1.034782   
b    one    1.0  0.597942       NaN  0.597942  0.597942  0.597942  0.597942   
     two    1.0 -0.423225       NaN -0.423225 -0.423225 -0.423225 -0.423225   

                    data2                                                    \
                max count      mean       std       min       25%       50%   
key1 key2                                                                     
a    one  -0.054101   2.0  0.381322  2.221313 -1.189383 -0.404030  0.381322   
     two  -1.034782   1.0 -1.034096       NaN -1.034096 -1.034096 -1.034096   
b    one   0.597942   1.0  0.505307       NaN  0.505307  0.505307  0.505307   
     two  -0.423225   1.0 -0.159151       NaN -0.159151 -0.159151 -0.159151   

                               
                75%       max  
key1 key2                      
a    one   1.166675  1.952028  
     two  -1.034096 -1.034096  
b    one   0.505307  0.505307  
     two  -0.159151 -0.159151

请注意下一条代码，虽然复杂，但极其好用

In [32]:
df.groupby(['key1','key2']).aggregate(['mean','std',lambda x:x.mean()]).rename(columns={'<lambda_0>':'mean_l'})

data1                         data2                    
               mean       std    mean_l      mean       std    mean_l
key1 key2                                                            
a    one  -0.447673  0.556595 -0.447673  0.381322  2.221313  0.381322
     two  -1.034782       NaN -1.034782 -1.034096       NaN -1.034096
b    one   0.597942       NaN  0.597942  0.505307       NaN  0.505307
     two  -0.423225       NaN -0.423225 -0.159151       NaN -0.159151

In [33]:
#事实上，更换名字并没有那么难
df.groupby(['key1','key2']).aggregate([('mean'),('std'),('mean_l',lambda x:x.mean())])

data1                         data2                    
               mean       std    mean_l      mean       std    mean_l
key1 key2                                                            
a    one  -0.447673  0.556595 -0.447673  0.381322  2.221313  0.381322
     two  -1.034782       NaN -1.034782 -1.034096       NaN -1.034096
b    one   0.597942       NaN  0.597942  0.505307       NaN  0.505307
     two  -0.423225       NaN -0.423225 -0.159151       NaN -0.159151

In [34]:
tips = pd.read_csv(wd+'//tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [35]:
tips.groupby(['day','smoker'])['tip'].aggregate(['mean','std'])

mean       std
day  smoker                    
Fri  No      2.812500  0.898494
     Yes     2.714000  1.077668
Sat  No      3.102889  1.642088
     Yes     2.875476  1.630580
Sun  No      3.167895  1.224785
     Yes     3.516842  1.261151
Thur No      2.673778  1.282964
     Yes     3.030000  1.113491

In [36]:
tips.groupby(['day','smoker'])['tip'].aggregate(['mean','std',('range',lambda x:x.max()-x.min())])

mean       std  range
day  smoker                           
Fri  No      2.812500  0.898494   2.00
     Yes     2.714000  1.077668   3.73
Sat  No      3.102889  1.642088   8.00
     Yes     2.875476  1.630580   9.00
Sun  No      3.167895  1.224785   4.99
     Yes     3.516842  1.261151   5.00
Thur No      2.673778  1.282964   5.45
     Yes     3.030000  1.113491   3.00

In [37]:
#事实上我们可以对不同变量做不同操作，这只需要输入一个字典即可
tips.groupby(['day','smoker'])[['tip','size']].aggregate({'tip':['min',('range',lambda x:x.max()-x.min())],
                                                        'size':['max']})

tip       size
              min range  max
day  smoker                 
Fri  No      1.50  2.00    3
     Yes     1.00  3.73    4
Sat  No      1.00  8.00    4
     Yes     1.00  9.00    5
Sun  No      1.01  4.99    6
     Yes     1.50  5.00    5
Thur No      1.25  5.45    6
     Yes     2.00  3.00    4

### Returning Aggregated Data Without Row Indexes

In [38]:
tips.groupby(['day','smoker'],as_index=False)[['tip','size']].aggregate({'tip':['min',('range',lambda x:x.max()-x.min())],
                                                        'size':['max']})

day smoker   tip       size
                 min range  max
0   Fri     No  1.50  2.00    3
1   Fri    Yes  1.00  3.73    4
2   Sat     No  1.00  8.00    4
3   Sat    Yes  1.00  9.00    5
4   Sun     No  1.01  4.99    6
5   Sun    Yes  1.50  5.00    5
6  Thur     No  1.25  5.45    6
7  Thur    Yes  2.00  3.00    4

## Apply: General split-apply-combine

In [39]:
tips.groupby('smoker').apply(lambda x:x.iloc[0:2,:])

total_bill   tip smoker  day    time  size
smoker                                               
No     0        16.99  1.01     No  Sun  Dinner     2
       1        10.34  1.66     No  Sun  Dinner     3
Yes    56       38.01  3.00    Yes  Sat  Dinner     4
       58       11.24  1.76    Yes  Sat  Dinner     2

### Example: Group Weighted Average

In [40]:
df = pd.DataFrame({'category':['a','a','a','a','b','b','b','b'],
                   'data':np.random.randn(8),
                   'weights':np.random.rand(8)})
df

,category,data,weights
0,a,2.095721,0.999663
1,a,-0.209006,0.681720
2,a,0.005575,0.187217
3,a,0.518296,0.370088
4,b,0.636188,0.211606
5,b,-1.324920,0.342507
6,b,1.054160,0.383258
7,b,-0.836567,0.881050


In [41]:
df.groupby('category').apply(lambda x:np.average(x['data'],weights=x['weights']))

category
a    0.958325
b   -0.358671
dtype: float64

In [42]:
close_px = pd.read_csv(wd+'//stock_px_2.csv', parse_dates=True,index_col=0)
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


In [43]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


### Example: Group-Wise Linear Regression

In [44]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1
    result = sm.OLS(Y,X).fit()
    return result

In [45]:
rets = close_px.pct_change().dropna()

In [46]:
rets.groupby(lambda x:x.year).apply(lambda x:sm.OLS(x['AAPL'],x['SPX'],).fit().params)

,SPX
2003,1.200841
2004,1.394995
2005,1.777145
2006,1.646548
2007,1.205186
2008,0.970660
2009,0.888944
2010,1.057903
2011,0.806352


## Pivot Tables and Cross-Tabulation

### pivot_table

pivot_table  

|Arguments|Description|
|---|---|
|values|
|index|
|columns| 
|aggfunc| 
|fill_value| 
|dropna|
|margins|
||

In [47]:
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3
240,27.18,2.00,Yes,Sat,Dinner,2
241,22.67,2.00,Yes,Sat,Dinner,2
242,17.82,1.75,No,Sat,Dinner,2


In [48]:
tips.pivot_table(['tip','size'],index=['time','day'],columns='smoker')

size                 tip          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  2.750000  3.003333
       Sat   2.555556  2.476190  3.102889  2.875476
       Sun   2.929825  2.578947  3.167895  3.516842
       Thur  2.000000       NaN  3.000000       NaN
Lunch  Fri   3.000000  1.833333  3.000000  2.280000
       Thur  2.500000  2.352941  2.666364  3.030000

In [49]:
tips.pivot_table(['tip','size'],index=['time','day'],columns='smoker',aggfunc=lambda x:len(x),margins=True,fill_value=0)

size           tip         
smoker        No Yes  All   No Yes  All
time   day                             
Dinner Fri     3   9   12    3   9   12
       Sat    45  42   87   45  42   87
       Sun    57  19   76   57  19   76
       Thur    1   0    1    1   0    1
Lunch  Fri     1   6    7    1   6    7
       Thur   44  17   61   44  17   61
All          151  93  244  151  93  244

### Cross-Tabulation

$$ pd.crosstab $$

$$ a^5 b^5 $$

